In [5]:
import pandas as pd
from datasets import Dataset
fce = pd.read_csv("./test_data/fce-18045.csv", sep=",")
CoNLL = pd.read_csv("./test_data/CoNLL-1195.csv", sep=",")
print(len(fce))
print(len(CoNLL))

18045
1195


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from peft import PeftModel

model = AutoModelForSeq2SeqLM.from_pretrained("./t5_grammar_corrector")
model = PeftModel.from_pretrained(model, "./t5_grammar_corrector")
tokenizer = AutoTokenizer.from_pretrained("./t5_grammar_corrector")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

d:\Hoctap\20242\NLP\PJ\venv\Lib\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
               

In [16]:
import nltk
nltk.data.path.append(r'C:\Users\84913\AppData\Roaming\nltk_data')
nltk.download('punkt', download_dir=r'C:\Users\84913\AppData\Roaming\nltk_data')

from nltk.translate.gleu_score import sentence_gleu
from nltk.tokenize import word_tokenize
import statistics
from tqdm import tqdm

def correct_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    outputs = model.generate(**inputs, max_length=128, num_beams=5)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# FCE
incorrect = fce["Incorrect"].tolist()
ground_truth = fce["Corrected"].tolist()

gleu_scores = []
predict = []

for i in tqdm(range(len(incorrect))):
    pred = correct_sentence(incorrect[i])
    predict.append(pred)

    ref_tokens = [word_tokenize(ground_truth[i].lower())]
    pred_tokens = word_tokenize(pred.lower())

    gleu = sentence_gleu(ref_tokens, pred_tokens)
    gleu_scores.append(gleu)

print("Mean GLEU:", statistics.mean(gleu_scores))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\84913\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
  0%|          | 0/18045 [00:00<?, ?it/s]

  0%|          | 0/18045 [00:03<?, ?it/s]


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\84913/nltk_data'
    - 'd:\\Hoctap\\20242\\NLP\\PJ\\venv\\nltk_data'
    - 'd:\\Hoctap\\20242\\NLP\\PJ\\venv\\share\\nltk_data'
    - 'd:\\Hoctap\\20242\\NLP\\PJ\\venv\\lib\\nltk_data'
    - 'C:\\Users\\84913\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\84913\\AppData\\Roaming\\nltk_data'
    - 'C:\\Users\\84913\\AppData\\Roaming\\nltk_data'
    - 'C:\\Users\\84913\\AppData\\Roaming\\nltk_data'
    - 'C:\\Users\\84913\\AppData\\Roaming\\nltk_data'
**********************************************************************
